In [1]:
import torch
torch.set_grad_enabled(False)
import soundfile as sf

import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().resolve().parent))
from models.generator import LCTGenerator, LCTGeneratorConfig
from util import resample_tensor, ModelComparator

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

d:\Projects\LCT-GAN\.venv\Lib\site-packages\df\io.py:9: UserWarning: `torchaudio.backend.common.AudioMetaData` has been moved to `torchaudio.AudioMetaData`. Please update the import path.
  from torchaudio.backend.common import AudioMetaData


In [2]:
jit = torch.jit.load("D:/Projects/LCT-GAN/.data/FTFNet_scripted.pt", map_location=device)
ftf = jit.model.eval()

my_gen = LCTGenerator(LCTGeneratorConfig()).to(device).eval()
my_gen.load_state_dict(ftf.state_dict(), strict=True)

cmp = ModelComparator(lct=jit, my_lct=my_gen, device=device)

In [3]:
noisy_path = "D:/Projects/LCT-GAN/.data/subjective_test_audios/impulse/noisy_fileid_1_snr14.25_tl-23.wav"
out_dir = Path("D:/Projects/LCT-GAN/.data/match_test/impulse")
out_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
x_np, sr_in = sf.read(noisy_path, dtype="float32")
if x_np.ndim > 1:
    x_np = x_np.mean(axis=1)

x = torch.from_numpy(x_np).unsqueeze(0).to(device)
noisy_16k = resample_tensor(x, sr_in, 16000)
noisy_16k = torch.clamp(noisy_16k, -1.0, 1.0)

with torch.no_grad():
    y_ftf = cmp.run_lct_gan(noisy_16k)
    y_my  = cmp.run_my_lct_gan(noisy_16k)

y_ftf = torch.clamp(y_ftf, -1.0, 1.0)
y_my  = torch.clamp(y_my,  -1.0, 1.0)

T = min(y_ftf.shape[-1], y_my.shape[-1])
diff = (y_ftf[..., :T] - y_my[..., :T]).abs()

print("max |FTFNet_wav - my_wav|:", diff.max().item())
print("mean |FTFNet_wav - my_wav|:", diff.mean().item())

sf.write(str(out_dir / "noisy_16k.wav"), noisy_16k.squeeze(0).detach().cpu().numpy(), 16000)
sf.write(str(out_dir / "ftfnet_16k.wav"), y_ftf.squeeze(0).detach().cpu().numpy(), 16000)
sf.write(str(out_dir / "my_ftfnet_16k.wav"), y_my.squeeze(0).detach().cpu().numpy(), 16000)
sf.write(str(out_dir / "diff_ftf_minus_my.wav"),
         (y_ftf[..., :T] - y_my[..., :T]).squeeze(0).detach().cpu().numpy(),
         16000)


max |FTFNet_wav - my_wav|: 0.5958443880081177
mean |FTFNet_wav - my_wav|: 0.04623037949204445


In [1]:
import torch
import torchaudio

import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().resolve().parent))
from models.generator import LCTGenerator, LCTGeneratorConfig
from datasets.stft import make_lct_stft, magnitude, apply_mask

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

noisy_wav_path = r"D:/Projects/LCT-GAN/.data/subjective_test_audios/impulse/noisy_fileid_1_snr14.25_tl-23.wav"
jit_path = r"D:/Projects/LCT-GAN/.data/FTFNet_scripted.pt"

# audio
wav, sr = torchaudio.load(noisy_wav_path)
wav = wav.mean(dim=0, keepdim=True)
if sr != 16000:
    wav = torchaudio.functional.resample(wav, sr, 16000)
wav = wav.to(device)                 # [1, N]
noisy = wav.unsqueeze(0)             # [B=1, 1, N]
noisy_bt = noisy.squeeze(1)          # [B, N]

# models
jit = torch.jit.load(jit_path, map_location=device)
ftf = jit.model.eval()

my_gen = LCTGenerator(LCTGeneratorConfig()).to(device).eval()
my_gen.load_state_dict(ftf.state_dict(), strict=True)

# STFT (project canonical)
stft = make_lct_stft(n_fft=512)
noisy_stft = stft(noisy_bt)                          # [B, F, T]
noisy_mag = magnitude(noisy_stft).unsqueeze(1)       # [B, 1, F, T]

# noisy_mag: [B, 1, F, T]
with torch.no_grad():
    mask_my_c = my_gen(noisy_mag)  # [B, 1, F, T]

    mask_ftf_c_TF = ftf(noisy_mag.permute(0, 1, 3, 2).contiguous())  # [B, 1, T, F] (typical)
    mask_ftf_c = mask_ftf_c_TF.permute(0, 1, 3, 2).contiguous()      # [B, 1, F, T]

    diff = (mask_ftf_c - mask_my_c).abs()
    print("max |mask_ftf_c - mask_my_c|:", diff.max().item())
    print("mean |mask_ftf_c - mask_my_c|:", diff.mean().item())


d:\Projects\LCT-GAN\.venv\Lib\site-packages\torchaudio\_backend\utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(


max |mask_ftf_c - mask_my_c|: 2.0696914196014404
mean |mask_ftf_c - mask_my_c|: 0.39401867985725403


In [5]:
print("FTF conv1:", ftf.conv1.kernel_size, ftf.conv1.stride, ftf.conv1.padding)
print("MY  conv1:", my_gen.conv1.kernel_size, my_gen.conv1.stride, my_gen.conv1.padding)

print("FTF conv2:", ftf.conv2.kernel_size, ftf.conv2.stride, ftf.conv2.padding)
print("MY  conv2:", my_gen.conv2.kernel_size, my_gen.conv2.stride, my_gen.conv2.padding)

print("FTF conv3:", ftf.conv3.kernel_size, ftf.conv3.stride, ftf.conv3.padding)
print("MY  conv3:", my_gen.conv3.kernel_size, my_gen.conv3.stride, my_gen.conv3.padding)

print("FTF deconv2:", ftf.deconv2.kernel_size, ftf.deconv2.stride, ftf.deconv2.padding, ftf.deconv2.output_padding)
print("MY  deconv2:", my_gen.deconv2.kernel_size, my_gen.deconv2.stride, my_gen.deconv2.padding, my_gen.deconv2.output_padding)

print("FTF deconv3:", ftf.deconv3.kernel_size, ftf.deconv3.stride, ftf.deconv3.padding, ftf.deconv3.output_padding)
print("MY  deconv3:", my_gen.deconv3.kernel_size, my_gen.deconv3.stride, my_gen.deconv3.padding, my_gen.deconv3.output_padding)

print("FTF deconv4:", ftf.deconv4.kernel_size, ftf.deconv4.stride, ftf.deconv4.padding, ftf.deconv4.output_padding)
print("MY  deconv4:", my_gen.deconv4.kernel_size, my_gen.deconv4.stride, my_gen.deconv4.padding, my_gen.deconv4.output_padding)


FTF conv1: (2, 3) (1, 2) (0, 0)
MY  conv1: (2, 3) (1, 2) (0, 0)
FTF conv2: (2, 3) (1, 2) (0, 0)
MY  conv2: (2, 3) (1, 2) (0, 0)
FTF conv3: (2, 3) (1, 2) (0, 0)
MY  conv3: (2, 3) (1, 2) (0, 0)
FTF deconv2: (2, 3) (1, 2) (0, 0) (0, 0)
MY  deconv2: (2, 3) (1, 2) (0, 0) (0, 0)
FTF deconv3: (2, 3) (1, 2) (0, 0) (0, 1)
MY  deconv3: (2, 3) (1, 2) (0, 0) (0, 1)
FTF deconv4: (2, 3) (1, 2) (0, 0) (0, 0)
MY  deconv4: (2, 3) (1, 2) (0, 0) (0, 0)


In [ ]:
print("ftf.activation:", ftf.activation)

ftf.activation: RecursiveScriptModule(original_name=LeakyReLU)


In [ ]:
import torch

def grab(name, store):
    def _hook(m, inp, out):
        store[name] = out.detach()
    return _hook

store_ftf = {}
store_my  = {}

# Choose comparable points
ftf.conv1.register_forward_hook(grab("conv1", store_ftf))
my_gen.conv1.register_forward_hook(grab("conv1", store_my))

# Also grab post-activation tensors by hooking conv2/conv3 etc. if needed
ftf.conv2.register_forward_hook(grab("conv2", store_ftf))
my_gen.conv2.register_forward_hook(grab("conv2", store_my))

with torch.no_grad():
    # Pick the same underlying TF input to conv1 in both models:
    x_FT = noisy_mag                       # [B,1,F,T] -> my_gen permutes internally
    x_TF = noisy_mag.permute(0,1,3,2).contiguous()  # [B,1,T,F] -> ftf public interface (typical)

    _ = my_gen(x_FT)
    _ = ftf(x_TF)

for k in ["conv1", "conv2"]:
    a = store_my[k]
    b = store_ftf[k]
    # conv outputs should match extremely closely if inputs/padding/weights match
    d = (a - b).abs()
    print(k, "max", d.max().item(), "mean", d.mean().item(), "shape", tuple(a.shape), tuple(b.shape))


RuntimeError: register_forward_hook is not supported on ScriptModules

In [ ]:
print(ftf)          # see if it has act_final / activations modules

RecursiveScriptModule(
  original_name=FTFNet
  (conv1): RecursiveScriptModule(original_name=Conv2d)
  (conv2): RecursiveScriptModule(original_name=Conv2d)
  (conv3): RecursiveScriptModule(original_name=Conv2d)
  (deconv2): RecursiveScriptModule(original_name=ConvTranspose2d)
  (deconv3): RecursiveScriptModule(original_name=ConvTranspose2d)
  (deconv4): RecursiveScriptModule(original_name=ConvTranspose2d)
  (activation): RecursiveScriptModule(original_name=LeakyReLU)
  (activations): RecursiveScriptModule(original_name=Sigmoid)
  (skip2): RecursiveScriptModule(original_name=Conv2d)
  (skip3): RecursiveScriptModule(original_name=Conv2d)
  (skip4): RecursiveScriptModule(original_name=Conv2d)
  (GRUf1): RecursiveScriptModule(
    original_name=GRUblockf
    (gru1): RecursiveScriptModule(original_name=GRU)
    (gru2): RecursiveScriptModule(original_name=GRU)
    (gru3): RecursiveScriptModule(original_name=GRU)
    (gru4): RecursiveScriptModule(original_name=GRU)
    (attn): RecursiveScript

In [6]:
print("activation slope:", ftf.activation.negative_slope)
print("GRUf1 activationtrans slope:", ftf.GRUf1.activationtrans.negative_slope)
print("GRUt1 activationtrans slope:", ftf.GRUt1.activationtrans.negative_slope)

print("layernorm eps:", ftf.layernorm.eps)

print("conv1:", ftf.conv1.kernel_size, ftf.conv1.stride, ftf.conv1.padding)
print("deconv3:", ftf.deconv3.kernel_size, ftf.deconv3.stride, ftf.deconv3.padding, ftf.deconv3.output_padding)

# To detect causal/trapezoid masking in scripted code:
print(ftf.GRUt1.code)      # or: print(ftf.GRUt1.graph)


activation slope: 0.03
GRUf1 activationtrans slope: 0.03
GRUt1 activationtrans slope: 0.03
layernorm eps: 1e-05
conv1: (2, 3) (1, 2) (0, 0)
deconv3: (2, 3) (1, 2) (0, 0) (0, 1)
def forward(self,
    x: Tensor) -> Tensor:
  b, c, t, f, = torch.size(x)
  _0 = torch.contiguous(torch.permute(x, [0, 3, 2, 1]))
  x22 = torch.view(_0, [torch.mul(b, f), t, c])
  feat_size = self.feat_size
  gru1 = self.gru1
  _1 = torch.slice(torch.slice(torch.slice(x22), 1), 2, None, torch.floordiv(feat_size, 4))
  gru_out1, _2, = (gru1).forward__0(_1, None, )
  gru2 = self.gru2
  _3 = torch.slice(torch.slice(x22), 1)
  _4 = torch.floordiv(feat_size, 4)
  _5 = torch.mul(torch.floordiv(feat_size, 4), 2)
  _6 = (gru2).forward__0(torch.slice(_3, 2, _4, _5), None, )
  gru_out2, _7, = _6
  gru3 = self.gru3
  _8 = torch.slice(torch.slice(x22), 1)
  _9 = torch.mul(torch.floordiv(feat_size, 4), 2)
  _10 = torch.mul(torch.floordiv(feat_size, 4), 3)
  _11 = (gru3).forward__0(torch.slice(_8, 2, _9, _10), None, )
  gru_o

In [7]:
print(ftf.graph)

graph(%self.1 : __torch__.lct_la1n.FTFNet,
      %x.1 : Tensor):
  %114 : NoneType = prim::Constant() # :0:0
  %51 : int = prim::Constant[value=0]() # /project_ghent/stress/lct_la1n.py:141:22
  %52 : int = prim::Constant[value=2]() # /project_ghent/stress/lct_la1n.py:141:25
  %53 : int = prim::Constant[value=3]() # /project_ghent/stress/lct_la1n.py:141:28
  %54 : int = prim::Constant[value=1]() # /project_ghent/stress/lct_la1n.py:141:31
  %100 : int = prim::Constant[value=-1]() # /project_ghent/stress/lct_la1n.py:150:34
  %121 : int = prim::Constant[value=-2]() # /project_ghent/stress/lct_la1n.py:154:34
  %142 : int = prim::Constant[value=-3]() # /project_ghent/stress/lct_la1n.py:158:34
  %pad.1 : __torch__.torch.nn.modules.padding.ConstantPad2d = prim::GetAttr[name="pad"](%self.1)
  %x0.1 : Tensor = prim::CallMethod[name="forward"](%pad.1, %x.1) # /project_ghent/stress/lct_la1n.py:131:12
  %7 : Tensor[] = prim::ListConstruct()
  %activation.1 : __torch__.torch.nn.modules.activation.Le